In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import normalize
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical


In [2]:
"""
Unloading the pickled data from process-data.ipynb
"""

pickle_data = pickle.load(open('picklefiles/training.pickle', 'rb'))

In [3]:
# Initialising empty lists for data
Xt = []
yt = []

In [4]:
#Adding the pickled data into the lists
for imgs, labels in pickle_data:
    Xt.append(imgs)
    yt.append(labels)

In [5]:
Xt = np.array(Xt)

In [6]:
yt = np.array(yt[0])

In [7]:
print(yt)


[1 1 1 ... 3 3 3]


In [8]:
""""
Scikit-learn train test split for dividing the data for training and testing.
"""
X_train,X_test,y_train,y_test = train_test_split(Xt,yt, test_size=0.1,random_state=101)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle=True)  

In [9]:
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

In [17]:
X_test

array([[[[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        ...,

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]

In [10]:
X_train.shape

(2757, 512, 512, 3)

In [11]:
print(y_train.shape)

(2757,)


In [12]:
y_train = to_categorical(y_train-1, 3);
y_test = to_categorical(y_test-1, 3);

In [13]:
model=Sequential([
    layers.Conv2D(16,4,padding="same",activation="relu",input_shape=(512,512,1)),
    layers.MaxPooling2D(),
    
    layers.Conv2D(32,4,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64,4,padding="same",activation="relu"),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    
    layers.Flatten(),
    
    layers.Dense(128,activation="relu"),
    layers.Dense(3)
])

In [14]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
history=model.fit(X_train,y_train ,epochs=5)

Epoch 1/5
87/87 [==============================] - 351s 4s/step - loss: 0.5977 - accuracy: 0.7320
Epoch 2/5
87/87 [==============================] - 343s 4s/step - loss: 0.2144 - accuracy: 0.9159
Epoch 3/5
87/87 [==============================] - 337s 4s/step - loss: 0.0889 - accuracy: 0.9710
Epoch 4/5
87/87 [==============================] - 333s 4s/step - loss: 0.0363 - accuracy: 0.9851
Epoch 5/5
87/87 [==============================] - 337s 4s/step - loss: 0.0228 - accuracy: 0.9924


In [20]:
model.save('saved-models')

INFO:tensorflow:Assets written to: saved-models\assets


INFO:tensorflow:Assets written to: saved-models\assets


In [25]:
from sklearn.metrics import classification_report,confusion_matrix

In [21]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

In [28]:
score = model.evaluate(X_test, y_test, verbose = 0 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])

Test Score:  0.22378498315811157
Test accuracy:  0.9155844449996948


In [26]:
print(classification_report(y_test_new,pred))

              precision    recall  f1-score   support

           0       0.76      0.95      0.84        37
           1       0.98      0.85      0.91        73
           2       0.98      1.00      0.99        44

    accuracy                           0.92       154
   macro avg       0.91      0.93      0.91       154
weighted avg       0.93      0.92      0.92       154

